<a href="https://colab.research.google.com/github/Efefefef/IANNwTF-UniOsnabrueck/blob/homework04/homework04_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
import tensorflow_datasets as tfds 
import tensorflow as tf

In [ ]:
def preprocess(data, task):
    batch_size = 32
    data = data.map(lambda image, target: (tf.cast(image, tf.float32) / 128. - 1, target))
    data = data.map(lambda image, target: (tf.reshape(image, (-1,)), target))
    #data = data.map(lambda image, target: (image, tf.one_hot(target, 10)))
    zipped_ds = tf.data.Dataset.zip((data.shuffle(2000), data.shuffle(2000))) # ((x1, y1), (x2, y2)
    
    # Mapping according to task 
    if task == 'subtask1': 
      # map ((x1, y1), (x2, y2)) to (x1, x2, x1+x2>=5)
      zipped_ds = zipped_ds.map(lambda x1, x2: (x1[0], x2[0], tf.cast((x1[1] + x2[1] >=5), tf.int32)))
    else: # task == 'subtask2'
      # map ((x1, y1), (x2, y2)) to (x1, x2, x1-x2)
      zipped_ds = zipped_ds.map(lambda x1, x2: (x1[0], x2[0], tf.cast((x1[1] - x2[1]), tf.int32)))
      zipped_ds = zipped_ds.map(lambda x1, x2, target: (x1, x2, tf.one_hot(target, 10)))
    
    zipped_ds.cache()
    zipped_ds = zipped_ds.shuffle(2000)
    zipped_ds = zipped_ds.batch(batch_size)
    zipped_ds = zipped_ds.prefetch(tf.data.AUTOTUNE)
    return zipped_ds

In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, optimiser):
        super(MyModel, self).__init__()
        self.metrics_list = [tf.keras.metrics.BinaryAccuracy(),
                        tf.keras.metrics.Mean(name="loss")]
        self.optimizer = optimiser
        self.loss_function = tf.keras.losses.BinaryCrossentropy()
        self.dense1 = tf.keras.layers.Dense(32, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(32, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)

    @tf.function
    def __call__(self, images, training=False):
        x, y = images

        x = self.dense1(x)
        x = self.dense2(x)

        y = self.dense1(y)
        y = self.dense2(y)

        z = self.out(x.concatenate(y))
        return z

    @property
    def metrics(self):
        return self.metrics_list

    def reset_metrics(self):
        for metric in self.metrics:
            metric.reset_states()
            
    def train_step(data):
        img1, img2, target = data

        with tf.GradientTape() as tape:
            prediction = self((img1, img2), training=True)
            loss = self.loss_function(target, prediction)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.metrics[0].update_state(target, prediction)
        self.metrics[1].update_state(loss)
        return {"accuracy": self.metrics[0], "loss": self.metrics[1]}

    def test_step(data):
        img1, img2, target = data
        prediction = self((img1, img2), training=False)
        loss = self.loss_function(target, prediction)
        self.metrics[0].update_state(target, prediction)
        self.metrics[1].update_state(loss)


In [ ]:
def training_loop(model, train_ds, test_ds, epochs, train_summary_writer, test_summary_writer, save_path):
    for epoch in range(epochs):
        model.reset_metrics()

        for data in train_ds:
            model.train_step(data)

        with train_summary_writer.as_default():
            tf.summary.scalar('accuracy', model.metrics[0].result(), step=epoch)
            tf.summary.scalar('loss', model.metrics[1].result(), step=epoch)

        model.reset_metrics()
        
        for data in test_ds:
            model.test_step(data)

        with test_summary_writer.as_default():
            tf.summary.scalar('accuracy', model.metrics[0].result(), step=epoch)
            tf.summary.scalar('loss', model.metrics[1].result(), step=epoch)

        print("Epoch: {}, Loss: {}, Accuracy: {}".format(epoch, model.metrics[1].result(), model.metrics[0].result()))

    model.save_weights(save_path)

In [ ]:
def train(subtask, optimiser):
    save_path = f"models/{subtask}_{optimiser}"
    train_log_path = f"logs/"
    test_log_path = f"logs/"
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_summary_writer = tf.summary.create_file_writer(train_log_path)
    test_summary_writer = tf.summary.create_file_writer(test_log_path)
    epochs = 10

    train_ds, test_ds = tfds.load('mnist', split=['train', 'test'], as_supervised=True)
    train_ds = preprocess(train_ds, subtask)
    test_ds = preprocess(test_ds, subtask)

    model = MyModel(optimiser)

    training_loop(model, train_ds, test_ds, epochs, train_summary_writer, test_summary_writer, save_path)

train('larger_than_five', tf.keras.optimizers.Adam())

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ~/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


TypeError: ignored

In [ ]:
train_ds, test_ds = tfds.load('mnist', split=['train', 'test'], as_supervised=True)
train_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
test = list(train_ds)[0]

AttributeError: ignored

In [ ]:
len(test)

2